In [1]:
import numpy as np
import matplotlib as plt

In [31]:
avo = 6.022e23
m2µm  = 1e6
def surf_a(d, l):
    return np.pi *  d  * l
def vol(d , l ):
    return np.pi*l*(d/2)**2

base_cyt = 10432667.412756/((3.1e-16 )*avo)   #moles ATP /sec per m^3 
base_mem = 909175.81/((5.68e-10 )*avo)      #moles ATP /sec per m^2

def base_cons(volume, sarea , bc=base_cyt, bm = base_mem ):
    return bc*volume + bm * sarea

In [27]:
base_cyt,base_mem

(0.05588469918233145, 2.658023310069651e-09)

In [28]:
d_range = np.arange(0,10,0.1)  # diameters in µm
l_range  = np.arange(0,10,0.1)  # length from mito midpoint to midpoint b/w  mito

In [36]:
base_cons(vol(d_range/m2µm,l_range/m2µm),surf_a(d_range/m2µm,l_range/m2µm))/vol(d_range/m2µm,l_range/m2µm)


C:\Users\ssempertegui2016\AppData\Local\Temp\ipykernel_1476\3425660271.py:1: RuntimeWarning: invalid value encountered in divide
  base_cons(vol(d_range/m2µm,l_range/m2µm),surf_a(d_range/m2µm,l_range/m2µm))/vol(d_range/m2µm,l_range/m2µm)


array([       nan, 0.16220563, 0.10904517, 0.09132501, 0.08246493,
       0.07714889, 0.07360485, 0.0710734 , 0.06917482, 0.06769814,
       0.06651679, 0.06555024, 0.06474478, 0.06406323, 0.06347905,
       0.06297276, 0.06252976, 0.06213887, 0.06179142, 0.06148054,
       0.06120075, 0.0609476 , 0.06071747, 0.06050735, 0.06031474,
       0.06013754, 0.05997397, 0.05982251, 0.05968188, 0.05955094,
       0.05942873, 0.05931441, 0.05920723, 0.05910655, 0.05901179,
       0.05892244, 0.05883806, 0.05875824, 0.05868262, 0.05861088,
       0.05854272, 0.05847789, 0.05841615, 0.05835728, 0.05830108,
       0.05824739, 0.05819602, 0.05814685, 0.05809972, 0.05805451,
       0.05801112, 0.05796942, 0.05792933, 0.05789075, 0.05785361,
       0.05781781, 0.05778329, 0.05774998, 0.05771782, 0.05768675,
       0.05765671, 0.05762767, 0.05759955, 0.05757233, 0.05754596,
       0.05752041, 0.05749562, 0.05747158, 0.05744824, 0.05742558,
       0.05740357, 0.05738218, 0.05736138, 0.05734115, 0.05732

C:\Users\ssempertegui2016\AppData\Local\Temp\ipykernel_1476\3425660271.py:1: RuntimeWarning: invalid value encountered in divide
  base_cons(vol(d_range/m2µm,l_range/m2µm),surf_a(d_range/m2µm,l_range/m2µm))/vol(d_range/m2µm,l_range/m2µm)


array([       nan, 0.16220563, 0.10904517, 0.09132501, 0.08246493,
       0.07714889, 0.07360485, 0.0710734 , 0.06917482, 0.06769814,
       0.06651679, 0.06555024, 0.06474478, 0.06406323, 0.06347905,
       0.06297276, 0.06252976, 0.06213887, 0.06179142, 0.06148054,
       0.06120075, 0.0609476 , 0.06071747, 0.06050735, 0.06031474,
       0.06013754, 0.05997397, 0.05982251, 0.05968188, 0.05955094,
       0.05942873, 0.05931441, 0.05920723, 0.05910655, 0.05901179,
       0.05892244, 0.05883806, 0.05875824, 0.05868262, 0.05861088,
       0.05854272, 0.05847789, 0.05841615, 0.05835728, 0.05830108,
       0.05824739, 0.05819602, 0.05814685, 0.05809972, 0.05805451,
       0.05801112, 0.05796942, 0.05792933, 0.05789075, 0.05785361,
       0.05781781, 0.05778329, 0.05774998, 0.05771782, 0.05768675,
       0.05765671, 0.05762767, 0.05759955, 0.05757233, 0.05754596,
       0.05752041, 0.05749562, 0.05747158, 0.05744824, 0.05742558,
       0.05740357, 0.05738218, 0.05736138, 0.05734115, 0.05732